In [1]:
using Pkg
Pkg.activate("C:\\Users\\Alex\\Desktop\\repo\\2612\\0903\\dynamical-systems\\env\\integrate\\")
using StaticArrays, ForwardDiff#, DifferentialEquations, DynamicalSystems

  Activating project at `C:\Users\Alex\Desktop\repo\2612\0903\dynamical-systems\env\integrate`


In [2]:
function TM(u, p, t)
    U(y) = p[8] + p[9] / ( 1.0 + exp( -50.0 * (y - p[7]) ) )
    σ(x) = 1.0 / ( 1.0 + exp( -20.0 * (x-p[6]) ) )
    du1 = (-u[1] + p[1] * log( 1.0 + exp( (p[5] * U(u[3]) * u[2] * u[1] + p[11]  ) / (p[1]) ) ) ) / p[2]
    du2 = (1.0 - u[2])/p[3] - U(u[3])*u[2]*u[1]
    du3 = (-u[3])/p[4] + p[10] * σ(u[2])
    return SA[du1, du2, du3]
end

TM (generic function with 1 method)

In [3]:
function jacob_TM_(u, p, t)
    U(y, p) = p[8] + p[9] / ( 1.0 + exp( -50.0 * (y - p[7]) ) )
    σ(x, p) = 1.0 / ( 1.0 + exp( -20.0 * (x - p[6]) ) )
    U_y(y, p) = ( 50.0 * p[9] * exp(-50.0 * (y - p[7])) ) / (1.0 + exp( -50.0*(y - p[7]) ) )^2
    g(E, x, y, p) = exp((p[5]  * U(y, p) * x * E + p[11]) / p[1])
    σ_der(x, p) = exp( (-20.0) * (x - p[6]) )
    E_E = ( -1.0 + ((J * U(u[3], p) * u[2] * g(u[1], u[2], u[3], p))) / (1.0 + g(u[1], u[2], u[3], p)) ) / p[2]
    E_x = (p[5] * U(u[3], p) * u[1] * g(u[1], u[2], u[3], p)) / (p[2] * (1.0 + g(u[1], u[2], u[3], p) ))
    E_y = ( p[5]  * U_y(u[3], p) * u[2] * u[1] * g(u[1], u[2], u[3], p) ) / (p[2] * (1.0 + g(u[1], u[2], u[3], p)) )
    
    x_E = -U(u[3], p)*u[2]
    x_x = -1.0 / p[3] - U(u[3], p)*u[1]
    x_y = (-U_y(u[3], p)) * u[2] * u[1]
    
    y_x = 20.0 * p[10] * σ_der(u[2], p) / (1.0 + σ_der(u[2], p))^2
    y_y = -1.0/p[4]
    
    SMatrix{3,3}(E_E, x_E, 0.0,
        E_x, x_x, y_x,
        E_y, x_y, y_y)
end

jacob_TM_ (generic function with 1 method)

In [14]:
const τ = 0.013;  const τD = 0.080;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4
const α = 1.58; const ΔU0 = 0.305;
U0 = 0.3; I0 = -1.8704

-2.6571

In [35]:
time = 250
transient_time = 250
tstep = 0.001
#integ_set = (alg = RK4(), adaptive = false, dt = tstep)

p = SA[α, τ, τD, τy, J, xthr, ythr, U0, ΔU0, β, I0]
u0 = SA[1.54672  0.930353  0.963849]
t = 0;

In [36]:
Jacobi(u0) = ForwardDiff.jacobian((x) -> TM(x, p, t), u0)

Jacobi (generic function with 1 method)

In [37]:
jacob_TM_(u0, p, t)

3×3 SMatrix{3, 3, Float64, 9} with indices SOneTo(3)×SOneTo(3):
 -10.1334    111.038      1.48529e-9
  -0.562864  -13.4358    -1.25171e-11
   0.0         0.154302  -0.30303

In [38]:
Jacobi(u0)

3×3 SMatrix{3, 3, Float64, 9} with indices SOneTo(3)×SOneTo(3):
 -10.1334    111.038      1.48529e-9
  -0.562864  -13.4358    -1.25171e-11
   0.0         0.154302  -0.30303

In [114]:
ds = CoupledODEs(TM,  u0, p, diffeq = integ_set)
tan = TangentDynamicalSystem(ds, J = jacob_TM_)
tan_auto  = TangentDynamicalSystem(ds)

3-dimensional TangentDynamicalSystem
 deterministic:     true
 discrete time:     false
 in-place:          false
 dynamic rule:      TM
 jacobian:          ForwardDiff
 deviation vectors: 3
 parameters:        [1.58, 0.013, 0.08, 3.3, 3.07, 0.75, 0.4, 0.3, 0.305, 0.3, -1.59]
 time:              0.0
 state:             [8.124338914021985, 0.6552902588595779, 0.4438414263930182]


In [115]:
@benchmark lyapunovspectrum(tan, time)

BenchmarkTools.Trial: 18 samples with 1 evaluation.
 Range (min … max):  277.929 ms … 287.031 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     281.196 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   281.640 ms ±   3.042 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

         █                             ▃                         
  ▇▇▁▁▇▇▁█▁▁▇▁▁▁▁▁▇▁▁▁▁▁▁▁▁▁▁▇▁▁▁▁▁▁▁▁▇█▇▁▁▁▁▁▁▁▇▇▁▁▁▁▁▁▁▁▇▁▁▁▇ ▁
  278 ms           Histogram: frequency by time          287 ms <

 Memory estimate: 3.28 KiB, allocs estimate: 47.

In [116]:
@benchmark lyapunovspectrum(tan_auto, time)

BenchmarkTools.Trial: 26 samples with 1 evaluation.
 Range (min … max):  195.002 ms … 201.918 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     195.455 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   196.571 ms ±   1.996 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▂ ▂                                                           
  ████▅▁▅▅▁▁▅▁▁▁▁▁▅▁▅▁▁▁▁▁▁▁▅▁▁▁▅▁▅█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅▁▁▁▅ ▁
  195 ms           Histogram: frequency by time          202 ms <

 Memory estimate: 3.28 KiB, allocs estimate: 47.